<!--
Copyright (c) 2025 Milin Patel
Hochschule Kempten - University of Applied Sciences

Autonomous Driving: AI Safety and Security
This project is licensed under the MIT License.
-->

*Copyright (c) 2025 Milin Patel. All Rights Reserved.*

# SOTIF Scenario Analysis and Triggering Conditions

**Module 04: Safety of the Intended Functionality (ISO 21448)**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/milinpatel07/Autonomous-Driving_AI-Safety-and-Security/blob/master/04_SOTIF/notebooks/02_scenario_analysis.ipynb)

**Author:** Milin Patel  
**Institution:** Hochschule Kempten - University of Applied Sciences

---

## Learning Objectives

By the end of this notebook, you will:
- Understand the SOTIF 4-quadrant scenario model
- Identify and classify triggering conditions for perception systems
- Learn systematic approaches for scenario generation
- Apply the scenario-based validation methodology
- Understand the relationship between scenarios and ODD boundaries

---

## 1. Introduction to SOTIF Scenarios

**ISO 21448** addresses safety issues arising from:
- Performance limitations of sensors and algorithms
- Reasonably foreseeable misuse by drivers
- Insufficient specification of functionality

Unlike ISO 26262 (which handles random hardware failures and systematic software bugs), SOTIF focuses on scenarios where the system works as designed but still produces unsafe behavior.

### The SOTIF Challenge

> "The system is not broken - it just cannot handle this scenario."

Examples:
- Camera cannot see pedestrian in low contrast conditions
- ML model misclassifies unusual vehicle type
- Sensor performance degrades in adverse weather

### References

This notebook builds on research in SOTIF analysis:
- Patel, M., Jung, R., Khatun, M. (2025). "A Systematic Literature Review on Safety of the Intended Functionality for Automated Driving Systems." SAE Technical Paper 2025-01-5030.
- ISO 21448:2022 - Road vehicles - Safety of the intended functionality

In [ ]:
# Setup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.patches import Rectangle, FancyBboxPatch, FancyArrowPatch
import seaborn as sns
from typing import Dict, List, Tuple
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')
print("Setup complete.")

## 2. The SOTIF 4-Quadrant Model

SOTIF categorizes scenarios into four quadrants based on two dimensions:

| | **Known** | **Unknown** |
|---|---|---|
| **Safe** | Area 1: Known Safe | Area 3: Unknown Safe |
| **Unsafe** | Area 2: Known Unsafe | Area 4: Unknown Unsafe |

### Quadrant Descriptions

- **Area 1 (Known Safe)**: Scenarios where system behaves safely, validated through testing
- **Area 2 (Known Unsafe)**: Identified hazardous scenarios, mitigated or restricted in ODD
- **Area 3 (Unknown Safe)**: Scenarios not explicitly tested but system handles safely
- **Area 4 (Unknown Unsafe)**: The dangerous unknown - hazardous scenarios not yet identified

### SOTIF Goal

**Minimize Area 4 (Unknown Unsafe)** by:
1. Discovering unknown scenarios through testing and analysis
2. Moving them to Area 2 (Known Unsafe) through identification
3. Mitigating or restricting them to become Area 1 (Known Safe)

In [ ]:
def visualize_sotif_quadrants():
    """Visualize the SOTIF 4-quadrant model with scenario flow."""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Left: Static quadrant model
    colors = {
        'A1': '#27ae60',  # Known Safe - Green
        'A2': '#e67e22',  # Known Unsafe - Orange  
        'A3': '#3498db',  # Unknown Safe - Blue
        'A4': '#e74c3c'   # Unknown Unsafe - Red
    }
    
    # Draw quadrants
    ax1.add_patch(Rectangle((0, 0.5), 0.5, 0.5, facecolor=colors['A1'], alpha=0.7))
    ax1.add_patch(Rectangle((0.5, 0.5), 0.5, 0.5, facecolor=colors['A3'], alpha=0.7))
    ax1.add_patch(Rectangle((0, 0), 0.5, 0.5, facecolor=colors['A2'], alpha=0.7))
    ax1.add_patch(Rectangle((0.5, 0), 0.5, 0.5, facecolor=colors['A4'], alpha=0.7))
    
    # Labels
    ax1.text(0.25, 0.75, 'Area 1\nKnown Safe', ha='center', va='center', 
            fontsize=12, fontweight='bold', color='white')
    ax1.text(0.75, 0.75, 'Area 3\nUnknown Safe', ha='center', va='center',
            fontsize=12, fontweight='bold', color='white')
    ax1.text(0.25, 0.25, 'Area 2\nKnown Unsafe', ha='center', va='center',
            fontsize=12, fontweight='bold', color='white')
    ax1.text(0.75, 0.25, 'Area 4\nUnknown Unsafe', ha='center', va='center',
            fontsize=12, fontweight='bold', color='white')
    
    # Axis labels
    ax1.set_xlabel('Knowledge Dimension', fontsize=12, fontweight='bold')
    ax1.set_ylabel('Safety Dimension', fontsize=12, fontweight='bold')
    ax1.set_xticks([0.25, 0.75])
    ax1.set_xticklabels(['Known', 'Unknown'])
    ax1.set_yticks([0.25, 0.75])
    ax1.set_yticklabels(['Unsafe', 'Safe'])
    ax1.set_xlim(0, 1)
    ax1.set_ylim(0, 1)
    ax1.set_title('SOTIF 4-Quadrant Model', fontsize=14, fontweight='bold')
    
    # Right: Scenario transition flow
    ax2.add_patch(FancyBboxPatch((0.1, 0.6), 0.25, 0.3, boxstyle="round,pad=0.02",
                                 facecolor=colors['A4'], alpha=0.8))
    ax2.add_patch(FancyBboxPatch((0.4, 0.6), 0.25, 0.3, boxstyle="round,pad=0.02",
                                 facecolor=colors['A2'], alpha=0.8))
    ax2.add_patch(FancyBboxPatch((0.7, 0.6), 0.25, 0.3, boxstyle="round,pad=0.02",
                                 facecolor=colors['A1'], alpha=0.8))
    
    ax2.text(0.225, 0.75, 'Unknown\nUnsafe', ha='center', va='center',
            fontsize=10, fontweight='bold', color='white')
    ax2.text(0.525, 0.75, 'Known\nUnsafe', ha='center', va='center',
            fontsize=10, fontweight='bold', color='white')
    ax2.text(0.825, 0.75, 'Known\nSafe', ha='center', va='center',
            fontsize=10, fontweight='bold', color='white')
    
    # Arrows
    ax2.annotate('', xy=(0.4, 0.75), xytext=(0.35, 0.75),
                arrowprops=dict(arrowstyle='->', lw=2, color='black'))
    ax2.annotate('', xy=(0.7, 0.75), xytext=(0.65, 0.75),
                arrowprops=dict(arrowstyle='->', lw=2, color='black'))
    
    # Process labels
    ax2.text(0.375, 0.5, 'Identification\n(Testing, Analysis)', ha='center', fontsize=9)
    ax2.text(0.675, 0.5, 'Mitigation\n(Design, ODD)', ha='center', fontsize=9)
    
    # Goal annotation
    ax2.text(0.5, 0.2, 'SOTIF Goal: Minimize Unknown Unsafe scenarios\n'
            'through systematic identification and mitigation',
            ha='center', fontsize=11, style='italic',
            bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))
    
    ax2.set_xlim(0, 1)
    ax2.set_ylim(0, 1)
    ax2.axis('off')
    ax2.set_title('SOTIF Scenario Transition Process', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()

visualize_sotif_quadrants()

## 3. Triggering Conditions

**Triggering conditions** are specific conditions that, when combined with a **functional insufficiency**, lead to a hazardous behavior.

### Structure

```
Triggering Condition + Functional Insufficiency → Hazardous Behavior → Harm
```

### Categories of Triggering Conditions

1. **Environmental Conditions**
   - Weather (rain, fog, snow, glare)
   - Lighting (day, night, tunnel transitions)
   - Road conditions (wet, icy, debris)

2. **Traffic Conditions**
   - Unusual road users (animals, wheelchairs)
   - Unexpected behavior (jaywalking, sudden stops)
   - Occlusions and limited visibility

3. **Infrastructure Conditions**
   - Road markings (faded, missing, contradictory)
   - Traffic signs (damaged, obscured)
   - Construction zones

4. **System Conditions**
   - Sensor degradation
   - Processing delays
   - Calibration drift

In [ ]:
# Triggering conditions taxonomy
triggering_conditions = {
    'Environmental': {
        'Weather': ['Heavy rain', 'Dense fog', 'Snow/Ice', 'Direct sunlight glare', 'Dust storm'],
        'Lighting': ['Nighttime', 'Dawn/Dusk', 'Tunnel entry/exit', 'Oncoming headlights', 'Shadows'],
        'Road Surface': ['Wet pavement', 'Ice/Snow', 'Debris', 'Potholes', 'Standing water']
    },
    'Traffic': {
        'Road Users': ['Motorcycles', 'Bicycles', 'Scooters', 'Animals', 'Wheelchairs/Mobility aids'],
        'Behavior': ['Jaywalking', 'Sudden lane change', 'Emergency vehicle', 'Wrong-way driver', 'Stationary vehicle'],
        'Occlusions': ['Parked vehicles', 'Large trucks', 'Vegetation', 'Buildings', 'Pedestrians behind objects']
    },
    'Infrastructure': {
        'Road Markings': ['Faded lines', 'Missing markings', 'Contradictory markings', 'Temporary markings', 'Worn paint'],
        'Signs/Signals': ['Damaged signs', 'Obscured signs', 'Malfunctioning signals', 'Temporary signs', 'Non-standard signs'],
        'Road Geometry': ['Sharp curves', 'Steep grades', 'Narrow lanes', 'Merging zones', 'Roundabouts']
    },
    'System': {
        'Sensor': ['Dirty lens', 'Sensor misalignment', 'Hardware degradation', 'EMI interference', 'Calibration drift'],
        'Processing': ['High latency', 'Memory overflow', 'Algorithm timeout', 'Communication delay', 'Data corruption'],
        'Model': ['Out-of-distribution input', 'Adversarial pattern', 'Domain shift', 'Rare class', 'Ambiguous scene']
    }
}

# Create comprehensive table
tc_list = []
for category, subcats in triggering_conditions.items():
    for subcat, conditions in subcats.items():
        for condition in conditions:
            tc_list.append({
                'Category': category,
                'Subcategory': subcat,
                'Triggering Condition': condition
            })

tc_df = pd.DataFrame(tc_list)
print(f"Total Triggering Conditions Identified: {len(tc_df)}")
print("\nDistribution by Category:")
print(tc_df['Category'].value_counts())
print("\nSample Triggering Conditions:")
display(tc_df.head(15))

In [ ]:
def visualize_triggering_conditions():
    """Visualize triggering condition categories."""
    fig, ax = plt.subplots(figsize=(12, 8))
    
    categories = list(triggering_conditions.keys())
    colors = ['#3498db', '#e74c3c', '#2ecc71', '#9b59b6']
    
    # Create sunburst-like visualization
    data = []
    for i, (cat, subcats) in enumerate(triggering_conditions.items()):
        total = sum(len(conditions) for conditions in subcats.values())
        data.append((cat, total))
    
    cats, values = zip(*data)
    
    # Outer ring - subcategories
    outer_colors = []
    outer_labels = []
    outer_values = []
    
    for i, (cat, subcats) in enumerate(triggering_conditions.items()):
        for subcat, conditions in subcats.items():
            outer_labels.append(f"{subcat}\n({len(conditions)})")
            outer_values.append(len(conditions))
            # Lighter shade of category color
            outer_colors.append(colors[i])
    
    # Plot
    inner_wedges, _ = ax.pie(values, radius=0.6, colors=colors,
                            wedgeprops=dict(width=0.3, edgecolor='white'))
    outer_wedges, texts = ax.pie(outer_values, radius=0.9, colors=outer_colors,
                                 labels=outer_labels, labeldistance=1.15,
                                 wedgeprops=dict(width=0.3, edgecolor='white', alpha=0.7),
                                 textprops={'fontsize': 8})
    
    # Legend
    legend_patches = [mpatches.Patch(color=colors[i], label=f"{cats[i]} ({values[i]})") 
                      for i in range(len(cats))]
    ax.legend(handles=legend_patches, loc='upper left', bbox_to_anchor=(1, 1))
    
    ax.set_title('Triggering Conditions Taxonomy\n(Number of conditions per category)', 
                fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()

visualize_triggering_conditions()

## 4. Functional Insufficiencies

**Functional insufficiencies** are limitations in the system's design that can lead to hazardous behavior when triggered.

### Categories for Perception Systems

| Category | Description | Examples |
|----------|-------------|----------|
| **Specification** | Incomplete or incorrect requirements | Missing edge case in ODD |
| **Sensor** | Physical limitations of sensors | Camera dynamic range, LiDAR range |
| **Algorithm** | Limitations of processing algorithms | ML model generalization |
| **Integration** | Issues in combining components | Sensor fusion timing |

In [ ]:
# Functional insufficiencies for perception systems
functional_insufficiencies = pd.DataFrame({
    'ID': ['FI-01', 'FI-02', 'FI-03', 'FI-04', 'FI-05', 'FI-06', 'FI-07', 'FI-08'],
    'Category': ['Sensor', 'Sensor', 'Algorithm', 'Algorithm', 'Specification', 
                 'Integration', 'Algorithm', 'Sensor'],
    'Description': [
        'Camera limited dynamic range',
        'LiDAR reduced performance in rain',
        'Object detector trained on limited dataset',
        'Tracking algorithm loses occluded objects',
        'ODD does not include construction zones',
        'Sensor fusion timing mismatch',
        'Classifier cannot distinguish pedestrian poses',
        'Radar cross-section varies with target angle'
    ],
    'Affected Function': [
        'Object detection in high contrast',
        '3D point cloud generation',
        'Rare object classification',
        'Multi-object tracking',
        'Safe operation boundaries',
        'Object position accuracy',
        'Pedestrian detection',
        'Vehicle detection'
    ],
    'Related Triggering Conditions': [
        'Direct sunlight, tunnel transitions',
        'Heavy rain, fog',
        'Unusual vehicles, animals',
        'Dense traffic, parked vehicles',
        'Construction zones',
        'High-speed scenarios',
        'Crouching, carrying objects',
        'Angled approaches'
    ]
})

print("Functional Insufficiencies for Perception Systems")
display(functional_insufficiencies)

## 5. Scenario Generation Methodology

Systematic scenario generation is essential for SOTIF validation. Based on research by Patel et al. (2024), we can use multiple approaches:

### Approaches

1. **Combinatorial** - Combine triggering conditions systematically
2. **Ontology-based** - Use domain knowledge to define scenario structure
3. **Data-driven** - Extract scenarios from real-world driving data
4. **Twin Scenarios** - Generate variations of known scenarios

### Reference

Patel, M., Jung, R., Khatun, M. (2024). "Assessing Unknown Hazards for SOTIF Based on Twin Scenarios Empowered Autonomous Driving." IEEE.

In [ ]:
class ScenarioGenerator:
    """Generate SOTIF scenarios from triggering conditions."""
    
    def __init__(self):
        self.base_scenarios = []
        self.generated_scenarios = []
    
    def add_base_scenario(self, name: str, description: str, 
                         functional_insufficiency: str):
        """Add a base scenario."""
        self.base_scenarios.append({
            'name': name,
            'description': description,
            'fi': functional_insufficiency,
            'triggering_conditions': []
        })
    
    def generate_combinatorial(self, scenario_idx: int, 
                               tc_categories: List[str]) -> List[Dict]:
        """Generate scenarios by combining triggering conditions."""
        if scenario_idx >= len(self.base_scenarios):
            return []
        
        base = self.base_scenarios[scenario_idx]
        scenarios = []
        
        # Get triggering conditions for specified categories
        tcs = []
        for cat in tc_categories:
            if cat in triggering_conditions:
                for subcat, conditions in triggering_conditions[cat].items():
                    for tc in conditions:
                        tcs.append(f"{subcat}: {tc}")
        
        # Generate scenario variants
        for i, tc in enumerate(tcs[:10]):  # Limit for demonstration
            scenario = {
                'ID': f"SC-{scenario_idx+1:02d}-{i+1:03d}",
                'Base': base['name'],
                'Triggering Condition': tc,
                'Functional Insufficiency': base['fi'],
                'Description': f"{base['description']} under {tc}"
            }
            scenarios.append(scenario)
            self.generated_scenarios.append(scenario)
        
        return scenarios
    
    def get_scenarios_df(self):
        return pd.DataFrame(self.generated_scenarios)

# Example: Generate scenarios
generator = ScenarioGenerator()

# Add base scenarios
generator.add_base_scenario(
    'Pedestrian Detection Failure',
    'System fails to detect pedestrian crossing road',
    'FI-01: Camera limited dynamic range'
)
generator.add_base_scenario(
    'Vehicle Tracking Loss',
    'System loses track of preceding vehicle',
    'FI-04: Tracking algorithm loses occluded objects'
)

# Generate scenarios for first base scenario
scenarios = generator.generate_combinatorial(0, ['Environmental', 'Traffic'])

print(f"Generated {len(scenarios)} scenarios for 'Pedestrian Detection Failure'")
display(generator.get_scenarios_df())

## 6. Scenario Classification

Each generated scenario must be classified into the 4-quadrant model based on:
- Whether it's been identified (Known vs Unknown)
- Whether it leads to safe or unsafe behavior

In [ ]:
def classify_scenario(scenario: Dict, 
                      known_conditions: List[str],
                      tested_conditions: List[str]) -> str:
    """Classify scenario into SOTIF quadrant."""
    tc = scenario['Triggering Condition']
    
    # Check if condition is known
    is_known = any(known in tc for known in known_conditions)
    
    # Check if condition has been tested (simplified logic)
    is_tested = any(tested in tc for tested in tested_conditions)
    
    # Determine safety (simplified - in practice requires simulation/testing)
    # Here we assume untested = potentially unsafe
    is_safe = is_tested
    
    if is_known and is_safe:
        return 'Area 1 (Known Safe)'
    elif is_known and not is_safe:
        return 'Area 2 (Known Unsafe)'
    elif not is_known and is_safe:
        return 'Area 3 (Unknown Safe)'
    else:
        return 'Area 4 (Unknown Unsafe)'

# Example classification
known_conditions = ['rain', 'fog', 'night', 'tunnel']
tested_conditions = ['rain', 'night']

# Classify generated scenarios
scenarios_df = generator.get_scenarios_df()
scenarios_df['Quadrant'] = scenarios_df.apply(
    lambda row: classify_scenario(row, known_conditions, tested_conditions), 
    axis=1
)

print("Scenario Classification Results")
print("=" * 50)
print(scenarios_df['Quadrant'].value_counts())
print("\nDetailed Classification:")
display(scenarios_df[['ID', 'Triggering Condition', 'Quadrant']])

In [ ]:
def visualize_scenario_classification(df):
    """Visualize scenario distribution across quadrants."""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    colors = {
        'Area 1 (Known Safe)': '#27ae60',
        'Area 2 (Known Unsafe)': '#e67e22',
        'Area 3 (Unknown Safe)': '#3498db',
        'Area 4 (Unknown Unsafe)': '#e74c3c'
    }
    
    # Pie chart
    counts = df['Quadrant'].value_counts()
    pie_colors = [colors.get(q, 'gray') for q in counts.index]
    ax1.pie(counts.values, labels=counts.index, colors=pie_colors,
           autopct='%1.0f%%', startangle=90,
           textprops={'fontsize': 10})
    ax1.set_title('Scenario Distribution by Quadrant', fontsize=12, fontweight='bold')
    
    # Bar chart with scenario IDs
    quadrant_order = ['Area 1 (Known Safe)', 'Area 2 (Known Unsafe)', 
                      'Area 3 (Unknown Safe)', 'Area 4 (Unknown Unsafe)']
    bar_colors = [colors[q] for q in quadrant_order if q in counts.index]
    existing_quadrants = [q for q in quadrant_order if q in counts.index]
    bar_values = [counts[q] for q in existing_quadrants]
    
    bars = ax2.bar(range(len(existing_quadrants)), bar_values, color=bar_colors, edgecolor='black')
    ax2.set_xticks(range(len(existing_quadrants)))
    ax2.set_xticklabels([q.split('(')[1].rstrip(')') for q in existing_quadrants], rotation=45, ha='right')
    ax2.set_ylabel('Number of Scenarios')
    ax2.set_title('Scenarios per Quadrant', fontsize=12, fontweight='bold')
    
    for bar, val in zip(bars, bar_values):
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
                str(val), ha='center', fontweight='bold')
    
    plt.tight_layout()
    plt.show()

visualize_scenario_classification(scenarios_df)

## 7. Validation Strategy

SOTIF validation aims to provide evidence that the residual risk from unknown unsafe scenarios is acceptable.

### Validation Methods

| Method | Purpose | Coverage |
|--------|---------|----------|
| **Simulation** | Test many scenarios quickly | Broad but limited fidelity |
| **Test Track** | Controlled physical testing | Medium coverage, high fidelity |
| **Field Testing** | Real-world driving | Limited scenarios, highest fidelity |
| **Analysis** | Expert review and formal methods | Supplements testing |

### Reference

Patel, M., Jung, R. (2024). "Simulation-Based Performance Evaluation of 3D Object Detection Methods with Deep Learning for a LiDAR Point Cloud Dataset in a SOTIF-related Use Case." VEHITS 2024.

In [ ]:
def visualize_validation_pyramid():
    """Visualize the SOTIF validation pyramid."""
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Pyramid layers (bottom to top)
    layers = [
        ('Simulation Testing\n(Millions of km)', 0.8, '#3498db', 
         'Scenario variation, stress testing, edge cases'),
        ('Test Track Validation\n(Thousands of km)', 0.6, '#2ecc71',
         'Physical testing, sensor performance, controlled conditions'),
        ('Field Operational Testing\n(Hundreds of km)', 0.4, '#f39c12',
         'Real-world exposure, unexpected scenarios'),
        ('Expert Analysis\n& Formal Methods', 0.25, '#9b59b6',
         'HARA, SOTIF analysis, safety case review')
    ]
    
    y_pos = 0.1
    for i, (label, width, color, description) in enumerate(layers):
        height = 0.18
        x_start = 0.5 - width/2
        
        # Draw trapezoid (simplified as rectangle)
        rect = FancyBboxPatch((x_start, y_pos), width, height,
                             boxstyle="round,pad=0.02",
                             facecolor=color, alpha=0.8, edgecolor='black')
        ax.add_patch(rect)
        
        # Label
        ax.text(0.5, y_pos + height/2, label, ha='center', va='center',
               fontsize=11, fontweight='bold', color='white')
        
        # Description on side
        ax.text(0.95, y_pos + height/2, description, ha='left', va='center',
               fontsize=9, style='italic')
        
        y_pos += height + 0.02
    
    # Arrows showing coverage vs fidelity
    ax.annotate('', xy=(0.05, 0.8), xytext=(0.05, 0.1),
               arrowprops=dict(arrowstyle='->', lw=2, color='gray'))
    ax.text(0.02, 0.45, 'Coverage', rotation=90, va='center', fontsize=10, fontweight='bold')
    
    ax.annotate('', xy=(0.12, 0.1), xytext=(0.12, 0.8),
               arrowprops=dict(arrowstyle='->', lw=2, color='gray'))
    ax.text(0.14, 0.45, 'Fidelity', rotation=90, va='center', fontsize=10, fontweight='bold')
    
    ax.set_xlim(0, 1.5)
    ax.set_ylim(0, 1)
    ax.axis('off')
    ax.set_title('SOTIF Validation Pyramid', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()

visualize_validation_pyramid()

## 8. Exercise: SOTIF Analysis

**Task:** Perform a SOTIF scenario analysis for a Lane Keeping Assist (LKA) system.

1. Identify at least 5 triggering conditions relevant to LKA
2. Define functional insufficiencies for LKA
3. Generate scenarios by combining triggering conditions with insufficiencies
4. Classify scenarios into the 4-quadrant model

In [ ]:
# Exercise: Your SOTIF analysis for Lane Keeping Assist

# TODO: Define LKA functional insufficiencies
lka_insufficiencies = [
    # Example: 'Camera cannot detect faded lane markings'
    # Add your insufficiencies here
]

# TODO: Identify relevant triggering conditions
lka_triggering_conditions = [
    # Example: 'Rain causing reflections on road'
    # Add your triggering conditions here
]

# TODO: Create scenario combinations
# Use the ScenarioGenerator class or create your own

print("Your LKA SOTIF Analysis:")
print(f"Functional Insufficiencies: {len(lka_insufficiencies)}")
print(f"Triggering Conditions: {len(lka_triggering_conditions)}")

## Summary

In this notebook, you learned:

- **SOTIF 4-Quadrant Model**: Framework for classifying scenarios by knowledge and safety
- **Triggering Conditions**: Environmental, traffic, infrastructure, and system conditions
- **Functional Insufficiencies**: Limitations in sensors, algorithms, and specifications
- **Scenario Generation**: Systematic methods for creating test scenarios
- **Validation Strategy**: Multi-level approach combining simulation, testing, and analysis

### Key Insight

> The goal of SOTIF is not to test everything, but to systematically reduce the unknown unsafe region through targeted identification and mitigation.

### Next Steps

- **Notebook 03**: SOTIF Validation Metrics and Acceptance Criteria
- **Module 02**: Out-of-Distribution Detection for identifying unknown scenarios
- **Module 06**: Uncertainty Quantification for safer AI perception

### References

- ISO 21448:2022 - Road vehicles - Safety of the intended functionality
- Patel, M., Jung, R., Khatun, M. (2025). "A Systematic Literature Review on SOTIF for ADS." SAE.
- Patel, M., Jung, R., Khatun, M. (2024). "Assessing Unknown Hazards for SOTIF." IEEE.
- Patel, M., Jung, R. (2024). "3D Object Detection in SOTIF Use Case." VEHITS.

---

*Notebook created by Milin Patel | Hochschule Kempten*  
*Last updated: 2025-01-22*